In [1]:
import tensorflow as tf
from tensorflow.python.training import optimizer
from tensorflow.python.framework import ops
from tensorflow.python.training import training_ops
from tensorflow.python.ops import math_ops

In [2]:
class MyOptimizer(optimizer.Optimizer):
    """A gradient descent optimizer which always use learning rate 1.0
    """
    
    def __init__(self, use_locking=True, name="MyOptimizer"):
        super(MyOptimizer, self).__init__(use_locking, name)
        self._learning_rate = 1.0
        
    def _prepare(self):
        self._learning_rate_tensor = ops.convert_to_tensor(self._learning_rate,
                                                           name="fuckup_learning_rate")
    
    def _apply_dense(self, grad, var):
        print("MyOptimizer._apply_dense!")
        return training_ops.apply_gradient_descent(var, 
                                                   math_ops.cast(self._learning_rate_tensor, var.dtype.base_dtype),
                                                   grad,
                                                   use_locking=self._use_locking)
    _apply_dense.__doc__ = optimizer.Optimizer._apply_dense.__doc__

## `Optimizer`

```
Optimizer.minimize`--> 
  Optimizer.compute_gradients`--> 
    Optimizer.apply_gradients`
```

In `Optimizer.apply_gradients`
1. it will call `self._prepare` method.
2. get `grad`, `var` and `processor (_RefVariableProcessor)` defined in `optimizer.py`
3. `processor` will use `processor.update_op` method to add update operation, which in turn call `optimizer`'s `_apply_dense` (if `var` are `Tensor`s) or `_apply_sparse_duplicate_indices` (if `var` are `IndexedSlices`).

In [3]:
print(MyOptimizer._apply_dense.__doc__)

Add ops to apply dense gradients to `var`.

    Args:
      grad: A `Tensor`.
      var: A `Variable` object.

    Return:
      An `Operation`.
    


In [4]:
graph = tf.Graph()
with graph.as_default():
    x = tf.Variable(0.5, dtype=tf.float32)
    loss = tf.abs(x)
    train_op = MyOptimizer().minimize(loss)

MyOptimizer._apply_dense!


In [5]:
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    for _ in range(3):
        sess.run(train_op)
        print(x.eval())

-0.5
0.5
-0.5
